# Parallel Error

In [1]:
class OnlyOne:
    class __OnlyOne:
        def __init__(self, arg):
            if arg is None:
                raise ValueError("Pretend empty instantiation breaks code")
            self.val = arg
        def __str__(self):
            return repr(self) + self.val
    instance = None
    def __init__(self, arg=None):
        if not self.instance:
            self.instance = self.__OnlyOne(arg)
        else:
            self.instance.val = arg
    def __getattr__(self, name):
        return getattr(self.instance, name)

In [2]:
one = OnlyOne(0)
print(one.val)

0


In [3]:
import time
from joblib import Parallel, delayed

def worker(num):
    one = OnlyOne()
    time.sleep(0.1)
    one.val += num
    return one.val


res = Parallel(n_jobs=-1, verbose=10)(delayed(worker)(i) for i in range(10))
print(res)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.5s remaining:    1.2s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    0.5s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.5s remaining:    0.1s


ValueError: Pretend empty instantiation breaks code

# Parallel Success

## Environment Variables

In [4]:
import os

class OnlyOne:
    class __OnlyOne:
        def __init__(self, arg):
            if arg is None:
                raise ValueError("Pretend empty instantiation breaks code")
            self.val = arg
        def __str__(self):
            return repr(self) + self.val
    instance = None
    def __init__(self, arg=None):
        if not self.instance:
            if arg is None:
                # look up val from env var
                arg = os.getenv('SINGLETON_VAL')
            else:
                # set env var so all workers use the same val
                os.environ['SINGLETON_VAL'] = arg
            self.instance = self.__OnlyOne(arg)
        else:
            self.instance.val = arg
    def __getattr__(self, name):
        return getattr(self.instance, name)

In [5]:
one = OnlyOne('abc')
print(one.val)

abc


In [8]:
print(os.getenv('SINGLETON_VAL'))

abc


In [7]:
import time
from joblib import Parallel, delayed

def worker(num):
    one = OnlyOne()
    time.sleep(0.1)
    one.val += str(num)
    return one.val


res = Parallel(n_jobs=-1, verbose=10)(delayed(worker)(i) for i in range(10))
print(res)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


['abc0', 'abc1', 'abc2', 'abc3', 'abc4', 'abc5', 'abc6', 'abc7', 'abc8', 'abc9']


[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.5s remaining:    1.4s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    0.5s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.5s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.7s finished


## Passing Singleton to Function

In [9]:
class OnlyOne:
    class __OnlyOne:
        def __init__(self, arg):
            if arg is None:
                raise ValueError("Pretend empty instantiation breaks code")
            self.val = arg
        def __str__(self):
            return repr(self) + self.val
    instance = None
    def __init__(self, arg=None):
        if not self.instance:
            self.instance = self.__OnlyOne(arg)
        else:
            self.instance.val = arg
    def __getattr__(self, name):
        return getattr(self.instance, name)

In [10]:
one = OnlyOne('abc')
print(one.val)

abc


In [11]:
import time
from joblib import Parallel, delayed

def worker(num, one):
    time.sleep(0.1)
    one.val += str(num)
    return one.val


res = Parallel(n_jobs=-1, verbose=10)(delayed(worker)(i, one) for i in range(10))
print(res)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


['abc0', 'abc1', 'abc2', 'abc3', 'abc4', 'abc5', 'abc6', 'abc7', 'abc8', 'abc9']


[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.6s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    0.6s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.6s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.7s finished
